In [1]:
import waterhealer as wh
import json

In [2]:
source = wh.from_kafka(
    ['testing'],
    {
        'bootstrap.servers': 'localhost:9095, localhost:9096',
        'group.id': 'group-test-2',
        'auto.offset.reset': 'latest',
    },
    debug = True
)

In [3]:
def json_loads(row):
    return (row[0], json.loads(row[1]))

def plus(row):
    no = row[1]['no']
    if no > 12:
        raise Exception('row is bigger than 12')
    return (row[0], no + 1)

In [4]:
outputs = []
source.map(json_loads).map(plus).map(wh.healing, stream = source).sink(outputs.append)

Output()

In [5]:
source.start()

In [6]:
from kafka import KafkaProducer
from kafka.partitioner import RoundRobinPartitioner

In [7]:
producer = KafkaProducer(
    bootstrap_servers = ['localhost:9095', 'localhost:9096'],
    api_version = (0, 10),
    partitioner = RoundRobinPartitioner(),
)

In [8]:
producer.bootstrap_connected()

True

In [9]:
from datetime import datetime
import json

def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding = 'utf-8')
        value_bytes = bytes(value, encoding = 'utf-8')
        x = producer_instance.send(topic_name, value = value_bytes)
        return True
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))
        return False

## purposely raise the error

Based on function `plus`, if input bigger than 10, it will throw error.

In [10]:
import confluent_kafka as ck

source.consumer.committed([ck.TopicPartition('testing', 0)])[0].offset

-1001

In [11]:
import time

for count in range(14):
    data = {'no': count, 'datetime': str(datetime.now())}
    print(data)
    publish_message(
        producer, 'testing', 'streaming', json.dumps(data)
    )
    time.sleep(1.0)

{'no': 0, 'datetime': '2020-04-07 18:56:50.461182'}


topic: testing, partition: 0, offset: 423, data: b'{"no": 0, "datetime": "2020-04-07 18:56:50.461182"}'
topic: testing, partition: 1, offset: 388, data: b'{"no": 1, "datetime": "2020-04-07 18:56:51.465847"}'


{'no': 1, 'datetime': '2020-04-07 18:56:51.465847'}


topic: testing, partition: 2, offset: 408, data: b'{"no": 2, "datetime": "2020-04-07 18:56:52.471373"}'


{'no': 2, 'datetime': '2020-04-07 18:56:52.471373'}


...

{'no': 11, 'datetime': '2020-04-07 18:57:01.508313'}


topic: testing, partition: 2, offset: 409, data: b'{"no": 12, "datetime": "2020-04-07 18:57:02.512270"}'


{'no': 12, 'datetime': '2020-04-07 18:57:02.512270'}


topic: testing, partition: 3, offset: 377, data: b'{"no": 13, "datetime": "2020-04-07 18:57:03.515344"}'
row is bigger than 12
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/streamz/core.py", line 578, in update
    result = self.func(x, *self.args, **self.kwargs)
  File "<ipython-input-3-f850c9be8189>", line 7, in plus
    raise Exception('row is bigger than 12')
Exception: row is bigger than 12
tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x11446ddd0>>, <Future finished exception=Exception('row is bigger than 12')>)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/tornado/ioloop.py", line 743, in _run_callback
    ret = callback()
  File "/usr/local/lib/python3.7/site-packages/tornado/ioloop.py", line 767, in _discard_future_result
    future.result()
  File "/usr/local/lib/python3.7/site-packages/tornado

{'no': 13, 'datetime': '2020-04-07 18:57:03.515344'}


If you check the logs,

```text
topic: testing, partition: 3, offset: 377, data: b'{"no": 13, "datetime": "2020-04-07 18:57:03.515344"}'
```

Offset for that message is 377, let's we check offset for that topic and partition.

In [12]:
source.consumer.committed([ck.TopicPartition('testing', 3)])[0].offset

377

Yeay, it is not updated! That is what we want!